# 총점 구하기

In [4]:
from pandas import DataFrame
from pandas import read_excel
import numpy as np

xlsx = read_excel('http://itpaper.co.kr/data/grade_card.xlsx', engine='openpyxl')
df = xlsx.set_index('이름')
df

,학년,성별,국어,영어,수학,과학
이름,,,,,,
철수,1,남자,98,NaN,88.0,64.0
영희,2,여자,88,90.0,62.0,72.0
민수,1,남자,92,70.0,NaN,NaN
수현,3,여자,63,60.0,31.0,70.0
호영,4,남자,120,50.0,NaN,88.0


## 합계점수에 대한 요약변수 만들기

In [5]:
df['총점'] = df['국어'] + df['영어'] + df['수학'] + df['과학']
df

,학년,성별,국어,영어,수학,과학,총점
이름,,,,,,,
철수,1,남자,98,NaN,88.0,64.0,NaN
영희,2,여자,88,90.0,62.0,72.0,312.0
민수,1,남자,92,70.0,NaN,NaN,NaN
수현,3,여자,63,60.0,31.0,70.0,224.0
호영,4,남자,120,50.0,NaN,88.0,NaN


# 파생변수
분석가의 노하우를 기반으로 새롭게 만들어낸 변수

- 사용자(분석자)가 특정 조건을 만족하거나 특정 함수에 의해 값을 만들어 의미를 부여한 변수.
- 기존의 변수(컬럼)을 조합하여 새로운 변수를 만들어 내는 것.
- 매우 주관적일 수 있으므로 논리적 타당성을 갖추어 개발해야 함.
- 세분화, 고객행동 예측, 캠페인 반응 예측에 자주 활용됨.
- 파생변수는 상황에 따라 특정 상황에만 유의미하지 않도록 대표성을 나타내야 함.

## 파생 변수를 위한 집계 함수

---

아래의 함수들은 기본적으로 열 단위로 집계를 수행한다. 행단위로 집계한 결과를 얻고자 할 경우 `axis=1` 파라미터를 설정해야 한다.

- 그룹별 데이터의 갯수: `size`, `count`
- 평균, 중앙값, 최소, 최대: `mean`, `median`, `min`, `max`
- 합계, 곱, 표준편차, 분산, 사분위수: `sum`, `prod`, `std`, `var`, `quantile`

In [6]:
# 점수만 추출
df2 = df.filter(['국어', '영어', '수학', '과학'])
df2

,국어,영어,수학,과학
이름,,,,
철수,98,NaN,88.0,64.0
영희,88,90.0,62.0,72.0
민수,92,70.0,NaN,NaN
수현,63,60.0,31.0,70.0
호영,120,50.0,NaN,88.0


In [7]:
# 열단위 합계 산출
df2.sum()

국어    461.0
영어    270.0
수학    181.0
과학    294.0
dtype: float64

In [8]:
# 열단위 평균 산출
df.mean()

학년      2.200000
국어     92.200000
영어     67.500000
수학     60.333333
과학     73.500000
총점    268.000000
dtype: float64

In [9]:
# 행 단위 합계와 평균을 파생변수로 추가

df3 = df2.copy()

df3['총정'] = df2.sum(axis = 1)

df3['평균'] = df2.mean(axis = 1)
df3

,국어,영어,수학,과학,총정,평균
이름,,,,,,
철수,98,NaN,88.0,64.0,250.0,83.333333
영희,88,90.0,62.0,72.0,312.0,78.000000
민수,92,70.0,NaN,NaN,162.0,81.000000
수현,63,60.0,31.0,70.0,224.0,56.000000
호영,120,50.0,NaN,88.0,258.0,86.000000


## 조건에 따른 선택적인 값으로 파생변수 추가
- numpy의 `where()` 함수를 사용하여 조건부 데이터 추가

In [11]:
# 조건이 참인 경우 A , 아닌경우 B
df3['결과'] = np.where(df3['평균'] >= 70, '합격', '불합격')
df3

,국어,영어,수학,과학,총정,평균,결과
이름,,,,,,,
철수,98,NaN,88.0,64.0,250.0,83.333333,합격
영희,88,90.0,62.0,72.0,312.0,78.000000,합격
민수,92,70.0,NaN,NaN,162.0,81.000000,합격
수현,63,60.0,31.0,70.0,224.0,56.000000,불합격
호영,120,50.0,NaN,88.0,258.0,86.000000,합격


In [12]:
# 여러 개의 조건중에서 선택적인 앖 추가
conditions = [
    (df3['평균'] >= 90), #A
    (df3['평균'] >= 80), #B
    (df3['평균'] >= 70), #C
    
]

# 조건에 따라 부여할 학점
grade = ['A', 'B', 'C']

df3['학점'] = np.select(conditions, grade, default='F')
df3

,국어,영어,수학,과학,총정,평균,결과,학점
이름,,,,,,,,
철수,98,NaN,88.0,64.0,250.0,83.333333,합격,B
영희,88,90.0,62.0,72.0,312.0,78.000000,합격,C
민수,92,70.0,NaN,NaN,162.0,81.000000,합격,B
수현,63,60.0,31.0,70.0,224.0,56.000000,불합격,F
호영,120,50.0,NaN,88.0,258.0,86.000000,합격,B
